### Load Dataset

In [ ]:
import pandas as pd

df_all = pd.read_csv('data/two_class.csv')
k = 1
n = df_all.shape[0]
df_test = df_all[round(n/5*(k-1)):round(n/5*k)]
df_train = pd.concat([df_all[0:round(n/5*(k-1))], df_all[round(n/5*k):n+1]])

### Train Word Vector

In [ ]:
wv_input = df_all['context'].map(lambda s: s.split(" "))   # [for w in s.split(" ") if w not in stopwords]                        

In [ ]:
from gensim import models

# Word2Vec
word2vec = models.Word2Vec(wv_input, 
                           vector_size=64,
                           min_count=1,
                           epochs=1000)

### BiLSTM

In [ ]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import numpy as np

#device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"

In [ ]:
# Super Character
learning_rate = 5e-4
input_size = 300
num_epoches = 8
batch_size = 10
embed_size = 64
hidden_size = 64
num_layers = 2

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = []
        self.label = []
        for t in df["emotion"].tolist():
            if t == 1:
                self.label.append(1)
            else:
                self.label.append(0)
        for s in df["context"].tolist():
            vectors = []
            for w in s.split(" "):
                if w in word2vec.wv.key_to_index:
                    vectors.append(word2vec.wv[w])   # replace word to vec
            vectors = torch.Tensor(vectors)
            self.data.append(vectors)
    
    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return data, label

    def __len__(self):
        return len(self.label)


def collate_fn(data):
    """
    :param data: data, label
    :return: vec-data、data length、label
    """
    data.sort(key=lambda x: len(x[0]), reverse=True) # reverse sort
    data_length = [len(sq[0]) for sq in data]
    x = [i[0] for i in data]
    y = [i[1] for i in data]
    data = pad_sequence(x, batch_first=True, padding_value=0)   # pad sequence
    return data, torch.tensor(y, dtype=torch.float32), data_length


# training data
train_data = MyDataset(df_train)
train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

# test data
test_data = MyDataset(df_test)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

In [ ]:
# Network Structure
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(hidden_size * 2, 1) 
        self.fc = nn.Linear(hidden_size, 1) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        #h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        #c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(input=x, lengths=lengths, batch_first=True)
        packed_out, (h_n, h_c) = self.lstm(packed_input, (h0, c0))

        lstm_out = torch.cat([h_n[-2], h_n[-1]], 1)
        #lstm_out = h_n[-1]
        out = self.fc(lstm_out)
        out = self.sigmoid(out)
        return out

lstm = LSTM(embed_size, hidden_size, num_layers)

In [ ]:
from sklearn import metrics

# Test
def test():
    y_pred, y_true = [], []

    with torch.no_grad():
        for x, labels, lengths in test_loader:
            x = x.to(device)
            outputs = lstm(x, lengths) 
            outputs = outputs.view(-1)
            y_pred.append(outputs)
            y_true.append(labels)

    y_prob = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    y_pred = []
    for s in y_prob:
        if s < 0.5:
            lab = 0
        else:
            lab = 1
        y_pred.append(lab)
    
    print("Accuracy score:", metrics.accuracy_score(y_true, y_pred))
    print("f-1 score:", metrics.f1_score(y_true, y_pred, average='macro'))

In [ ]:
# Loss and Optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [ ]:
# Training
for epoch in range(num_epoches):
    total_loss = 0
    for i, (x, labels, lengths) in enumerate(train_loader):
        x = x.to(device)
        labels = labels.to(device)
        outputs = lstm(x, lengths) 
        logits = outputs.view(-1)  
        loss = criterion(logits, labels)   
        total_loss += loss
        optimizer.zero_grad()              
        loss.backward(retain_graph=True)    
        optimizer.step()                    
        if (i+1) % 10 == 0:
            total_loss = 0
    test()